In [ ]:
import pandas as pd

# Importing from the web
Import the data from the Wikipedia table at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

I use `pd.read_html` to transform the HTML table into a dataframe.
The content of each cell is then transferred to the mapping function `getCellData` that splits the postal code, the borough and the neighbourhood name. An intermediate list (array) of dictionaries is used to create the final dataframe.

The dataframe is then cleaned by getting rid of the rows that contain undefined boroughs. 

In [ ]:
origin_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Pandas already allows to parse HTML tables within a web page; # we do not need
# this information categorized in a dataframe yet, but we can use the dataframe
# to parse the table and get all the postal codes. 
# We flatten the table and apply a filter to it to clean each cell's content; 
# Postal codes are composed of three characters, which correspond to
# the first three of each cell.

# Cell processing function
def getCellData(x):
  x = x.strip()
  r = {'postalCode': x[0:3]}
  if x.find(')') > -1:
    r['Borough'] = x[3:x.index('(')]
    r['Neighbourhood'] = x[x.index('(')+1:x.index(')')].replace('/', ',').replace(' ,', ',')
    return r
  else:
    r['Borough'] = x[3::]
  return r

# From web to pandasa and applying processor function
postal_codes = pd.read_html(origin_url)[0] \
  .applymap( getCellData ) \
  .stack().values

# List of dicts to dataframe
postal_codes = pd.DataFrame.from_records(postal_codes)

# Drop rows with unassigned borough
postal_codes = postal_codes[ postal_codes['Borough'] != 'Not assigned' ]

# Set the unassigned neighbourhood to be the same as the borough
postal_codes.loc[postal_codes['Neighbourhood'].isnull(), ['Neighbourhood']] = \
postal_codes.loc[postal_codes['Neighbourhood'].isnull(), ['Borough']] 

# Shape & show
print("Shape:", postal_codes.shape)
postal_codes.head()

Shape: (103, 3)


,postalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [ ]:
# Done!